# Optimize 3d conv

First results:   
<pre>
python perf_convs.py --inputs 20 20 20 20 3 --filters 7 3 6 2 3      
Input shape  [20, 20, 20, 20, 3]
Filter shape [7, 3, 6, 2, 3]
Output shape (20, 18, 15, 19, 7)
#Inputs   480000
#Weights     756
#Outputs  718200
#Multiplications 542959200
Theano 3d                 runtime per iteration (  14 iterations): 143.8815ms
Theano 3d2d               runtime per iteration (  78 iterations):  12.9532ms
Python Vectorized         runtime per iteration (  25 iterations):  82.0083ms
Cudamat Vectorized        runtime per iteration (  23 iterations):  87.3997ms
Theano Mul Vectorized     runtime per iteration (  28 iterations):  72.9542ms
GPU Loop                  runtime per iteration (   1 iterations): 20829.8290ms


</pre>

After Cudnn theano 3d2d faster:

<pre>
python perf_convs.py --inputs 20 20 20 20 3 --filters 7 3 6 2 3
Using gpu device 0: GeForce GTX 780
Batches/Filters, rows, columns, times, channels
Input shape  [20, 20, 20, 20, 3]
Filter shape [7, 3, 6, 2, 3]
Output shape (20, 18, 15, 19, 7)
#Inputs   480000
#Weights     756
#Outputs  718200
#Multiplications 542959200

Theano 3d                 runtime per iteration (  14 iterations): 143.8013ms
Theano 3d2d               runtime per iteration ( 286 iterations):   3.5014ms
Python Vectorized         runtime per iteration (  25 iterations):  80.9831ms
Cudamat Vectorized        runtime per iteration (  24 iterations):  86.5586ms
Theano Mul Vectorized     runtime per iteration (  28 iterations):  71.6942ms
GPU Loop                  runtime per iteration (   1 iterations): 20715.0171ms

</pre>

In [1]:
# 80 x 80 x 3 image input with 5 - 40 frames batch size 64, 256
# 5 x  5 x 3 x 5(zeit) filter, 128 davon
# wenn im vergleich yuzu 2d nur um faktor zeit oder ein bisschen mehr groesser dann ok

In [2]:
# wrap as a layer usable in theano:  https://github.com/benanne/Lasagne/blob/master/lasagne/layers/corrmm.py

In [ ]:
# or pylearn2

## Debug

In [2]:
import os
os.sys.path.insert(0,'/home/schirrmr/3dconv/') 
%cd /home/schirrmr/3dconv/vol_conv/
%load_ext autoreload
%autoreload 2

/home/schirrmr/3dconv/vol_conv


In [8]:
!python tests/test_training.py

Using gpu device 0: GeForce GTX 780
/home/schirrmr/3dconv/pylearn2-master/pylearn2/utils/image.py:16: UserWarning: Unable to import matplotlib. Some features unavailable. Original exception: No module named matplotlib.pyplot
  "Original exception: " + str(matplotlib_exception))
Parameter and initial learning rate summary:
	conv3d_lin_W: 0.10000000149
	conv3d_lin_b: 0.10000000149
	softmax_b: 0.10000000149
	softmax_W: 0.10000000149
Compiling sgd_update...
Compiling sgd_update done. Time elapsed: 2.804602 seconds
0.58
0.92
0.84
1.0


In [3]:
from vol_conv.volumetric_space import Conv3DSpace
from vol_conv.layers.theano_3d_conv import Theano3dConv3dElemwise
import numpy as np
from pylearn2.models.mlp import IdentityConvNonlinearity
import theano
import theano.tensor as T
from vol_conv.volumetric_dense_design_matrix import VolumetricDenseDesignMatrix
from pylearn2.training_algorithms.sgd import SGD
from pylearn2.models.mlp import MLP, Softmax
from pylearn2.format.target_format import OneHotFormatter
from numpy.random import RandomState
class FakeMLP():
    def __init__(self,rng,batch_size):
        self.rng = rng
        self.batch_size = batch_size

Using gpu device 0: GeForce GTX 780
/home/schirrmr/3dconv/pylearn2-master/pylearn2/utils/image.py:16: UserWarning: Unable to import matplotlib. Some features unavailable. Original exception: No module named matplotlib.pyplot
  "Original exception: " + str(matplotlib_exception))


In [4]:

fake_mlp = FakeMLP(rng=rng,batch_size=inputs_shape[0])
conv_3d_layer = Theano3dConv3dElemwise(output_channels=filters_shape[0], 
        kernel_shape=filters_shape[1:4],
        layer_name='conv3d_lin', nonlinearity=IdentityConvNonlinearity(),
        irange=0.001)
conv_3d_layer.set_mlp(fake_mlp)
conv_3d_input_space = Conv3DSpace(inputs_shape[1:4], num_channels=inputs_shape[4], axes=('b',0,1,2,'c'))
conv_3d_layer.set_input_space(conv_3d_input_space)


inputs_3d_layer_theano = ftensor5()
conv3d__layer_result = conv_3d_layer.fprop(inputs_3d_layer_theano)
conv3d_fprop = theano.function([inputs_3d_layer_theano], conv3d__layer_result)

inputs_3d_nnet_theano = ftensor5()
filters_3d_nnet_theano = ftensor5()
bias_3d_nnet_theano = T.fvector()
conv3d_nnet_result = theano.tensor.nnet.conv3D(inputs_3d_nnet_theano, filters_3d_nnet_theano, bias_3d_nnet_theano, d=(1,1,1))
conv3d_nnet = theano.function([inputs_3d_nnet_theano, filters_3d_nnet_theano, bias_3d_nnet_theano], conv3d_nnet_result)

inputs = (rng.rand(*inputs_shape).astype('float32') -1) * 2 # [-1,1)
filters = (rng.rand(*filters_shape).astype('float32') - 1) *2 # [-1,1)
bias = rng.rand(filters.shape[0]).astype('float32')
targets = np.zeros(inputs.shape[0]).astype('int')
targets[::2] = 1 # every second target is 1 or 0
inputs[targets == 1] = inputs[targets == 1] + 1


correct_result = conv3d_nnet(inputs, filters, bias)
conv_3d_layer.set_weights(filters)
conv_3d_layer.set_biases(bias)
layer_result = conv3d_fprop(inputs)

assert np.sum(np.square(correct_result - layer_result)) < 1e-4

target_formatter = OneHotFormatter(2)
targets_one_hot = target_formatter.format(targets)

conv_3d_layer.mlp = None
softmax_layer = Softmax(max_col_norm=2, layer_name='y', n_classes=2, istdev=.05)
mlp = MLP(input_space=conv_3d_input_space, layers=[conv_3d_layer, softmax_layer])
train_set = VolumetricDenseDesignMatrix(topo_view=inputs[0:50], y=targets_one_hot[0:50], axes=('b', 0, 1, 2, 'c'))
valid_set = VolumetricDenseDesignMatrix(topo_view=inputs[50:75], y=targets_one_hot[50:75], axes=('b', 0, 1, 2, 'c'))
test_set = VolumetricDenseDesignMatrix(topo_view=inputs[75:100], y=targets_one_hot[75:100], axes=('b', 0, 1, 2, 'c'))
algorithm = SGD(batch_size=20, learning_rate=0.1)
algorithm.setup(mlp, train_set)

Parameter and initial learning rate summary:
	conv3d_lin_W: 0.10000000149
	conv3d_lin_b: 0.10000000149
	softmax_b: 0.10000000149
	softmax_W: 0.10000000149
Compiling sgd_update...
Compiling sgd_update done. Time elapsed: 2.951343 seconds


In [5]:
inputs_mlp_theano = ftensor5()
mlp_fprop_result = mlp.fprop(inputs_mlp_theano)

mlp_fprop = theano.function([inputs_mlp_theano], mlp_fprop_result)
mlp_result = mlp_fprop(train_set.get_topological_view())
mlp_result_labels = np.argmax(mlp_result, axis=1)
y_labels = np.argmax(train_set.y, axis=1)
np.sum(np.equal(y_labels, mlp_result_labels)) / float(len(train_set.y))

0.29999999999999999

In [6]:
algorithm.train(train_set)

In [7]:
mlp_valid_result = mlp_fprop(valid_set.get_topological_view())
mlp_valid_result_labels = np.argmax(mlp_valid_result, axis=1)
mlp_test_result = mlp_fprop(test_set.get_topological_view())
mlp_test_result_labels = np.argmax(mlp_test_result, axis=1)
print("valid", np.sum(np.equal(np.argmax(valid_set.y, axis=1), mlp_valid_result_labels)) / float(len(valid_set.y)))
print("test", np.sum(np.equal(np.argmax(test_set.y, axis=1), mlp_test_result_labels)) / float(len(test_set.y)))

('valid', 0.88)
('test', 0.95999999999999996)


In [ ]:
# make test case with this data, check that you always get same results
#

# for pyyaml later:
# create h5 files with data, create "h5volumetricdataset" class which reads out adata and calls superconstructor
